In [1]:
#Import libraries

import numpy as np
from numpy import argmax
import os
import matplotlib.pyplot as plt
import pickle
import math

from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from keras.utils import to_categorical
from keras.utils import multi_gpu_model
from keras.utils import Sequence
from keras.models import load_model
from tensorflow.distribute import MirroredStrategy

In [2]:
#Define Data gemerator

rng = np.random.default_rng()
class FelixSequence(Sequence):
    def __init__(self, x_set, y_set, batch_size, file_type):
        """Here self.x is a list of paths to file_type files. self.y is a
        corresponding list of labels."""
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.file_type = file_type

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return arrs_from_paths(batch_x, self.file_type), to_categorical(np.array(batch_y),10)

def gen_paths_labels(base_path = "D:\\Uni Work\\Masters Project\\test_dir"):
    """A generator to yield (data-paths, corresponding labels) tuples for each
    segment of data (typically training, validation, and testing)."""
    for segment in sorted(os.listdir(base_path)):
        segment_path = os.path.join(base_path, segment)
        segment_paths = []
        segment_labels = []
        for label in os.listdir(segment_path):
            label_path = os.path.join(segment_path, label)
            for crystal in os.listdir(label_path):
                segment_paths.append(os.path.join(label_path, crystal))
                segment_labels.append(label)
        indexes = np.arange(len(segment_labels))
        rng.shuffle(indexes)
        yield [np.array(segment_paths)[indexes], np.array(list(map(int,segment_labels)))[indexes]]

def arrs_from_paths(paths, file_type):
    if file_type == "txt":
        return np.array([np.loadtxt(file_name) for file_name in paths])
    elif file_type == "npy":
        return np.array([np.load(file_name) for file_name in paths])

In [3]:
#All paths

Path = "/home/ug-ml/felix-ML/DataGenerator2/Data" #Path where training and validation data is
SaveDataPath = "/home/ug-ml/Documents/GitHub_BigFiles/SaveFolder" #Base directory of place you store information of models
SaveFolderName = "/Convnet2" #Will create a folder and put in information about the outcome / inputs
ModelName = "/Convnet2.hdf5"

batch_size = 32

In [4]:
#Load data generators

data = [i for i in gen_paths_labels(Path)]
#data[0][0] is training paths data[0][1] is training labels
#data[1][0] is val paths data[1][1] is val labels
print("data done.")
training_seq = FelixSequence(data[0][0], data[0][1], batch_size, "npy")
print("training_seq done.")
#print(data[1][0])
val_images = arrs_from_paths(data[1][0], "npy")
print("val_images done.")
val_lab = to_categorical(data[1][1])
print("val_lab done.")
print("Complete")

data done.
training_seq done.
val_images done.
val_lab done.
Complete


In [5]:
model = load_model(SaveDataPath + SaveFolderName + ModelName)
model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d_18 (Separab (None, 128, 126, 126)     5060      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 128, 63, 63)       0         
_________________________________________________________________
separable_conv2d_19 (Separab (None, 128, 61, 61)       17664     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 128, 30, 30)       0         
_________________________________________________________________
separable_conv2d_20 (Separab (None, 128, 28, 28)       17664     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 128, 14, 14)       0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 25088)            

In [6]:
#Run model

history = model.fit(training_seq, 
                    epochs=1, 
                    validation_data = (val_images, val_lab), 
                    #callbacks=[EarlyStop, Checkpoint], 
                    workers = 16,
                    shuffle = True,
                    use_multiprocessing = False)


ResourceExhaustedError:  OOM when allocating tensor with shape[32,36,126,126] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential_6/separable_conv2d_18/separable_conv2d/depthwise (defined at <ipython-input-6-11c7c1693853>:9) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_933]

Function call stack:
train_function
